## 単純なCNNモデルの作成

もしも```ModuleNotFoundError: No module named 'keras'```
といったエラーが出力された場合は
```!pip install keras```
を実行してanacondaにkerasをインストールしてください。

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                       input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3),activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3),activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3),activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))

Using TensorFlow backend.


In [ ]:
model.summary()

### モデルのコンパイル

In [ ]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

### ImageDataGeneratorを使ってディレクトリから画像を読み込む

In [ ]:
import pandas as pd
import os
base_dir = "mini_pictures"
train_dir = os.path.join(base_dir,"train")
valid_dir = os.path.join(base_dir,"valid")
test_dir = os.path.join(base_dir,"test")
mini_metadata = pd.read_csv('mini_metadata.csv',index_col=0)
classes = mini_metadata["make_model"].value_counts().index.tolist()
print(classes)
# mini_metadata.groupby("make_model").nunique()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#すべての画像を1/255スケーリング
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#flow_from_directoryではcategoricalのラベリングはclassesを参照して、ディレクトリ名から自動的にone-hotエンコードされる
train_generator = train_datagen.flow_from_directory(
    train_dir,               #ターゲットディレクトリ
    target_size=(150, 150),  #すべての画像サイズを150*150に変換
    batch_size=20,           #バッチサイズ
    class_mode='categorical',
    classes=classes)#categorical_crossentropyを使用するため多クラスラベルが必要

validation_generator = test_datagen.flow_from_directory(
    valid_dir,               #ヴァリデーションディレクトリ
    target_size=(150, 150),  #すべての画像サイズを150*150に変換
    batch_size=20,           #バッチサイズ
    class_mode='categorical',
    classes=classes)#categorical_crossentropyを使用するため多クラスラベルが必要


### バッチジェネレータを使ってモデルを適応

In [ ]:
history = model.fit_generator(train_generator,
                             steps_per_epoch=100,
                             epochs=30,
                             validation_data=validation_generator,
                             validation_steps=50)

In [ ]:
model.save("simplest_cnn_model_mini.h5")

In [ ]:
import matplotlib.pylot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(acc)+1)

#正解率をプロット
plt.plot(epohs, acc, 'bo', label='Training acc')
plt.plot(epohs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')

plt.figure()

#損失値をプロット
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validatin loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## データ拡張
### ImageDataGeneratorを使ってデータ拡張を設定する

In [ ]:
datagen = ImageDataGenerator(rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')